In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv("../input/nyc-taxi-trip-duration/train.zip")
test = pd.read_csv("../input/nyc-taxi-trip-duration/test.zip")

In [ ]:
# Количество отсутствующих данных
print(f"na sum of train: {train.isna().sum()}")

print(f"na sum of tst: {test.isna().sum()}")

In [ ]:
# Разбиваем Datetime на более подробные данные
train['pickup_datetime'] = pd.to_datetime(train['pickup_datetime'])

train['pickup_hour'] = [d.hour for d in train['pickup_datetime']]
train['pickup_week'] = [d.week for d in train['pickup_datetime']]
train['pickup_month'] = [d.month for d in train['pickup_datetime']]
train['pickup_weekdays'] = [d.weekday() for d in train['pickup_datetime']]
train['pickup_weekend'] = ["Not-weekend" for d in train['pickup_datetime']]
train.loc[train['pickup_weekdays'].isin([5, 6]), 'pickup_weekend'] = "Weekend"

train

In [ ]:
train['mday'] = [d.day for d in train['pickup_datetime']]
train['yday'] = [d.dayofyear for d in train['pickup_datetime']]
train

In [ ]:
# Расчитываем расстояние поездки
from pyproj import Geod

wgs84_geod = Geod(ellps='WGS84') #Distance will be measured on this ellipsoid - more accurate than a spherical method


def Distance(lat1, lon1, lat2, lon2):
    az12,az21,dist = wgs84_geod.inv(lon1,lat1,lon2,lat2) #Yes, this order is correct
    return dist

    
pickups = [train['pickup_longitude'], train['pickup_latitude']]
dropoffs = [train['dropoff_longitude'], train['dropoff_latitude']]
train['distance_m'] = Distance(train['pickup_latitude'].tolist(), train['pickup_longitude'].tolist(), train['dropoff_latitude'].tolist(), train['dropoff_longitude'].tolist())
train['distance_km'] = train['distance_m'] / 1000

train['distance_km']

In [ ]:
# Подготовка тестового набора
test['pickup_datetime'] = pd.to_datetime(test['pickup_datetime'])
test['pickup_hour'] = [d.hour for d in test['pickup_datetime']]
pickups = [test['pickup_longitude'], test['pickup_latitude']]
dropoffs = [test['dropoff_longitude'], test['dropoff_latitude']]
test['distance_m'] = Distance(test['pickup_latitude'].tolist(), test['pickup_longitude'].tolist(), test['dropoff_latitude'].tolist(), test['dropoff_longitude'].tolist())
test['distance_km'] = test['distance_m'] / 1000
test

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
x = train[['pickup_hour', 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']]
y = train[['trip_duration']]
lr.fit(x, y)

In [ ]:
x = test[['pickup_hour', 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']].copy()
x = np.array(x.values)
print(x)
y = pd.Series(np.array(lr.predict(x)).ravel())

print(y.mean())

In [ ]:
lr.coef_